In [6]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [7]:
#  !pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_google_genai

In [8]:
from crewai import Agent, Task, Crew
from langchain_google_genai import ChatGoogleGenerativeAI
import os


In [9]:
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
os.environ["SERPER_API_KEY"] = userdata.get("SERPER_API_KEY")

# Initialize Gemini-2.5 Pro LLM
gemini_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0.3,
    top_p=0.95,
)

In [10]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	    memory=True,
  llm=gemini_llm,
	verbose=True
)

In [11]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True,
	 memory=True,
  llm=gemini_llm,
)

In [12]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

In [13]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

In [14]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

In [15]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


In [16]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=2,


)

In [17]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


> Entering new CrewAgentExecutor chain...
Thought:
The user, Andrew Ng from DeepLearningAI, is asking for help with setting up a Crew and specifically how to add memory to it. This is a super important customer, so I need to provide the best possible support.

My plan is to:
1.  Use the `Read website content` tool to access the official documentation on creating and kicking off a crew. The URL provided in the tool description seems perfect for this: `https://docs.crewai.com/how-to/Creating-a-Crew-and

In [18]:
from IPython.display import Markdown
Markdown(result)

Hello Andrew,

Thank you so much for reaching out to us at crewAI! It's an absolute pleasure to assist you and the team at DeepLearningAI. I'm a huge admirer of your work, and we're committed to providing you with the best support possible.

You asked for guidance on setting up a Crew and, specifically, how to add memory to it. That's an excellent question, as memory is a crucial component for enabling sophisticated collaboration and context-awareness among your agents.

To ensure I provided the most accurate and up-to-date information, I consulted our official documentation, specifically the page on [Creating a Crew and kicking it off](https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/). While this page offers a fantastic high-level overview of the crewAI framework, its components, and how to get started, the specific implementation details for adding memory are located in other sections of our documentation.

Not to worry, I can absolutely provide you with the detailed guidance you're looking for!

### Understanding Memory in crewAI

In crewAI, memory allows a crew to maintain context throughout its execution. When memory is enabled, each agent can access the history of previous actions, observations, and the outcomes of tasks completed by other agents. This is vital for complex projects where context is key. It prevents agents from repeating work and enables them to build upon each other's findings, leading to a more coherent and intelligent final output.

### How to Add Memory to Your Crew

Adding memory to your crew is straightforward. It's done during the instantiation of your `Crew` object by setting the `memory` parameter to `True`.

When you set `memory=True`, you are activating a powerful short-term memory system for your crew. Under the hood, crewAI leverages a Retrieval-Augmented Generation (RAG) approach. The output of each completed task is embedded and stored in a local, in-memory vector database. When a subsequent agent begins its task, it uses the task description and context to query this vector store, retrieving the most relevant information from the work of previous agents.

Here is a code example to illustrate how you can implement this:

```python
# First, make sure you have crewAI installed
# pip install crewai

from crewai import Agent, Task, Crew, Process

# Define your agents with their roles, goals, and backstories
# For example, a researcher agent
researcher = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge developments in AI and data science',
  backstory="""You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting
  actionable insights.""",
  verbose=True,
  allow_delegation=False
  # You can add tools to your agent here if needed
  # tools=[some_tool]
)

# And a writer agent
writer = Agent(
  role='Tech Content Strategist',
  goal='Craft compelling content on tech advancements',
  backstory="""You are a renowned Content Strategist, known for
  your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
  verbose=True,
  allow_delegation=True
)

# Create tasks for your agents
task1 = Task(
  description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.""",
  expected_output="A full analysis report in markdown format.",
  agent=researcher
)

task2 = Task(
  description="""Using the insights provided, develop an engaging blog post that highlights
  the most significant AI advancements. Your post should be informative yet accessible,
  catering to a tech-savvy audience. Make it sound cool, avoid complex words so it doesn't
  sound like AI.""",
  expected_output="A full blog post of at least 4 paragraphs in markdown format.",
  agent=writer
)

# Instantiate your crew with memory enabled
# This is the key step for adding memory
my_crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  process=Process.sequential,  # Defines the execution flow of tasks
  memory=True,               # <-- THIS IS THE MAGIC PARAMETER!
  cache=True,                # Caches tool outputs for efficiency
  max_rpm=100,               # Maximum requests per minute to avoid rate limiting
  verbose=2                  # Provides detailed execution logs for debugging
)

# Kick off the crew's work!
result = my_crew.kickoff()

print("######################")
print("## Here is the result:")
print("######################")
print(result)
```

### Understanding the Crew Configuration

Let's break down a few of the key configuration parameters from the example above to give you full control over your crew's execution.

*   **`process`**: This parameter dictates how your tasks are executed. You have two main options:
    *   **`Process.sequential`**: This is the most common process. Tasks are executed one by one, in the exact order you list them. The output of the first task is passed as context to the second, the second to the third, and so on. It's perfect for linear workflows where each step builds directly on the last.
    *   **`Process.hierarchical`**: This process is designed for more complex, dynamic workflows. It requires a "manager" agent (an agent with `allow_delegation=True`). The manager receives the initial tasks, analyzes them, and can delegate them to other agents, create new sub-tasks, and orchestrate the overall workflow. It's best for situations that require planning and dynamic task management, rather than a fixed, step-by-step process.

### Key Takeaways for Fine-Tuning Your Crew:

*   **`memory=True`**: This is the primary switch to enable contextual memory for your crew's execution. It ensures that the knowledge generated by one task is available to all subsequent tasks within a *single run*.
*   **`cache=True`**: While not memory in the same sense, this is another useful parameter. It caches the results of tool calls with the same inputs. This can save time and costs on repeated actions, especially if your agents use the same tool with the same arguments multiple times within a single execution.
*   **`verbose`**: This parameter is incredibly useful for debugging and understanding what your crew is doing "under the hood." It can be set to `True` (or `1`) for a high-level overview, or `2` for maximum detail. With `verbose=2`, the output will show you each agent's thought process, the actions it considers, the specific tools it uses, the inputs to those tools, and the observations it makes. This detailed log is invaluable for diagnosing issues or refining agent instructions.
*   **Long-Term Memory (Persistence Across Runs)**: The `memory=True` parameter enables short-term memory that is cleared after each `kickoff()` is complete. For long-term memory that persists across different crew executions, you would need to integrate an external, persistent vector store (like ChromaDB, Weaviate, Pinecone, etc.). The best way to achieve this in crewAI is by creating a custom tool that allows your agents to save and retrieve information from this database.

    Here is a conceptual example of how you might create a `MemoryTool` using a library like ChromaDB. This tool would allow an agent to explicitly save key findings for future crews to access.

    ```python
    # Conceptual Example: A custom tool for long-term memory
    # You would need to install chromadb: pip install chromadb
    import chromadb
    from crewai_tools import BaseTool

    # Initialize your persistent vector database
    client = chromadb.PersistentClient(path="/path/to/your/db")
    # This could be a singleton or managed in a better way in a real app
    memory_collection = client.get_or_create_collection(name="long_term_memory")

    class MemoryTool(BaseTool):
        name: str = "Long-Term Memory"
        description: str = "Stores or retrieves key information from a persistent database. Use 'save' to store information and 'retrieve' to search for it."

        def _run(self, operation: str, content: str) -> str:
            if operation == 'save':
                # Save content to ChromaDB
                # The 'id' could be a hash of the content or a timestamp
                doc_id = str(hash(content))
                memory_collection.add(
                    documents=[content],
                    ids=[doc_id]
                )
                return f"Information saved with ID: {doc_id}"
            elif operation == 'retrieve':
                # Retrieve relevant information from ChromaDB
                results = memory_collection.query(
                    query_texts=[content],
                    n_results=5 # Get top 5 most relevant results
                )
                return str(results['documents'])
            else:
                return "Error: Operation must be 'save' or 'retrieve'."

    # You would then add this tool to your agent:
    # memory_tool = MemoryTool()
    # researcher.tools = [memory_tool]
    ```
    By giving an agent this tool, you empower it to decide what information is critical to save for the long term. For more details on creating your own tools, our documentation on [Custom Tools](https://docs.crewai.com/how-to/Creating-Custom-Tools/) is a great resource.

I hope this detailed explanation and code example are helpful for you and the DeepLearningAI team! We are truly excited to see what you build with crewAI.

If you have any more questions or need further clarification, please don't hesitate to ask. We're here to help you succeed.

Best regards,

Your Senior Support Representative at crewAI